## Теоретическая часть


1. Ответьте на вопросы:  
В чем принципиальное отличие гибридных рекомендательных систем от коллаборативной филтьтрации?  
Приведите 2-3 примера задач, в которых необходимо использовать гибридные системы  




Ответы

1. Коллаборативная фильтрация может быть частью гибридной РС, которая может включать в себя кроме этого вида фильтрации, ещё и контентную фильтрацию, либо какую либу иную форму фильтрации. Кроме того, коллаборативная требует наличия исторических данных о товаре или клиенте, а гибридная может этого избежать, если изначально будет рассматривать контентный анализ. Таким образом можно избавиться от проблемы "хоролдного старта"

2. Netflix - гибрид. Amazon в начале своей деятельности - это популяризация коллаборативной фильтрации, которая здесб дыла полностью испробована и давала неплохие результаты.

2.  Прочитайте статью про поиск на hh.ru https://habr.com/ru/company/hh/blog/347276/
Нам интересна именно рекомендательная система, раздел "Производительность системы" можно пропустить
Какие основные отличия предложенной системы от тех подходов, которые мы разбирали на семинарах? Какие проблемы могут возникнуть при выводе такой модели в продакшен?

Ответ

1. гибридную систему где сначало идет контентный фильтр для признаков по вакансиям с помощью библиотеки Lucene. 
Затем расчет признаков по методу пар (paitwise)
Потом фильрующая модель logit (с удлучшением и отладкой модели за счте подкачки offline метрик).
Потом ранжирующая модель (XGBoost) 
Потом выгрузка результатов клиенту и A/B-тесты
Всё это зацикливается и как-то должно работать.

Проблемы две: вычислительные мощности и ручное измерение offline-метрик. С обеими проблемами нужно явно вести борьбу в данной общей гибридной модели. 

3. На вебинаре мы рассматривали модель LightFM (https://making.lyst.com/lightfm/docs/lightfm.html). В работе Data Scientist'а важную часть занимает research - исследование существующих архитектур и разбор научных статей, в которых они описываются. Вам предлагается изчуть оригинальную статью про LightFM https://arxiv.org/pdf/1507.08439.pdf и ответить на следующие вопросы:  
1) Какой датасет используют авторы?  
2) Что используют в качестве признаков?  
3) С какими моделями сравнивают LightFM? Опишите их основные идеи кратко  

Ответ:
    
    1) Огромный каталог модной одежды на 8 млн позиций (+ десятки тысяч позиций обновляются каждый день).
    Разбит на 2 датасета (DS): один охватывают диапазон данных плотного взаимодействия юзера с товарами, где можно ожидать, что модели матричной факторизации (MF) будут работать хорошо (название DS - MovieLens), и разреженных данных, где контентые модели (CB), будут работать скорее всего лучше (DS - CrossValidated). 
    
    2) В DS MovieLens: юзеры, товары, взаимодействия, уникальные теги.
    В DS CrossValidated: юзеры + метаданные "Обо мне", вопросы + уникальные теги, ответы и комментарии.
    
    3) Для сравнения с LightFM автором брались следующие можели: 
    
    1. MF: традиционная модель матричной факторизации со смещением пользователя и элемента и сигмовидной функцией связи [8].

2. LSI-LR: модель на основе контента. Чтобы оценить его, я сначала вывожу скрытые темы из матрицы элементов-функций с помощью латентной семантической индексации и представляю элементы как линейные комбинации скрытых тем. Затем я подбираю отдельную модель логистической регрессии (LR) для каждого пользователя в пространстве смешения тем. В отличие от модели LightFM, которая использует совместные данные для создания своего скрытого представления, LSI-LR основана исключительно на факторизации матрицы контента. Поэтому это должно быть полезно для выделения преимуществ использования совместной информации для построения вложений функций.

3. LSI-UP: гибридная модель, которая представляет профили пользователей (UP) как линейные комбинации векторов содержимого элементов, а затем применяет LSI к результирующей матрице для получения скрытых представлений пользователей и элементов. Я оцениваю эту модель, сначала создавая матрицу характеристик пользователя: каждая строка представляет пользователя и задается суммой векторов характеристик контента, представляющих элементы, с которыми пользователь положительно взаимодействовал. Затем я применяю усеченный SVD к нормализованной матрице, чтобы получить скрытые векторы пользователей и признаков; Скрытые векторы элементов получаются путем их проецирования на пространство скрытых признаков. Таким образом, оценка рекомендаций для пары «пользователь-элемент» является внутренним продуктом их скрытых представлений.

4. LightFM (теги): модель LightFM, использующая только функции тегов.

5. LightFM (теги + идентификаторы): модель LightFM, использующая функции тегов и индикаторов элементов.

6. LightFM (теги + информация): модель LightFM, использующая как предметные, так и пользовательские функции. Пользовательские функции доступны только для набора данных CrossValidated. Я создаю их, преобразовывая разделы «Обо мне» профилей пользователей в набор слов. Сначала я удаляю из них все HTML-теги и неалфавитные символы, затем преобразую полученную строку в нижний регистр и размечаю пробелами.
    

## Практическая часть


In [ ]:
import lightfm

C:\ProgramData\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:10: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  "LightFM was compiled without OpenMP support. "


In [ ]:
# import data
# utils functions like in webinar

### 1. Модуль SRC

На вебинаре было рассказано про модуль src. Он приложен в материалах. Скачайте его, изучите структуру, импортируйте функции

### 2. Работа с признаками

У нас есть внешние данные. Что с ними не так? Чего не хватает?  

Проведите исследование внешних данных и составьте какие-нибудь содержательные выводы.  
Формально Вам нужно построить 3+ графиков (scatter plot, hist или что-то иное) и описать, что мы видим (например, товары такой-то категории болле часто покупаются в следующие дни недели или пользователи с большим достатком предпочитают такие-то товары).  
Исследуйте те закономерности, которые Вам интересно, чем менее тривиальный вывод получается, тем лучше! 

### 3. LightFM

У этого алогритма есть множество параметров (item/user_alpha, loss, no_components).  
Проведите эксперименты аналогично дз 3 (подберите гипперпараметры каким удобно способои и постройте графики)  
На выходе необходимо получить pr@5 на валидации (последние 3 недели) > 2%  

У Вас, скорее всего, возникнет проблема со временем обучения. Почему они возникает?    

Попробуйте запустить алгоритм вообще без фичей или используйте только признаки с небольшим числом уникальных категорий. (item_features['commodity_desc'].unique() - 300 уникальных категорий - это очень много)

### *Отбор признаков* * 

Все данные категориальные, при ohe кодировании для товаров признаков становится невероятно много.      
Какие стратегии отбора признаков в классическом ML Вы знаете? Применимы ли они тут?  

Попробйте какие-нибудь стратегии. Удалось ли улучшить качество?

 \* *задание необязательно*


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from utils import prefilter_items

ModuleNotFoundError: No module named 'src'

In [4]:
data = pd.read_csv('retail_train.csv')

item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [6]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [7]:
n_items_train_before = data_train['item_id'].nunique()
n_items_test_before = data_test['item_id'].nunique()
n_users_train_before = data_train['user_id'].nunique()
n_users_test_before = data_test['user_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)
data_test_filtered = prefilter_items(data_test, take_n_popular=5000, item_features=item_features)

n_items_train_after = data_train_filtered['item_id'].nunique()
n_items_test_after = data_test_filtered['item_id'].nunique()
n_users_train_after = data_train_filtered['user_id'].nunique()
n_users_test_after = data_test_filtered['user_id'].nunique()
print('Decreased # items_train from {} to {}'.format(n_items_train_before, n_items_train_after))
print('Decreased # items_test from {} to {}'.format(n_items_test_before, n_items_test_after))
print('Decreased # users_train from {} to {}'.format(n_users_train_before, n_users_train_after))
print('Decreased # users_test from {} to {}'.format(n_users_test_before, n_users_test_after))

NameError: name 'prefilter_items' is not defined

In [8]:
user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

NameError: name 'data_train_filtered' is not defined

In [11]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit
test_user_item_matrix.head(2)

NameError: name 'data_test_filtered' is not defined

In [12]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

NameError: name 'user_item_matrix' is not defined

In [ ]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

In [ ]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

In [ ]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())b

In [ ]:
user_feat_lightfm.head(2)

In [ ]:
model = LightFM(no_components=40,
                 loss='bpr',
                #loss='warp',
                learning_rate=0.05, 
                item_alpha=0.1,
                user_alpha=0.1, 
                random_state=42)

In [ ]:
%%time

model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15, 
          num_threads=4,
          verbose=False) 

In [ ]:
train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

train_precision


In [ ]:
test_precision = precision_at_k(model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

test_precision

In [ ]:
!pip install hyperopt

In [ ]:
from hyperopt import tpe, hp, fmin, STATUS_OK, Trials

space = {
    'no_components': hp.choice('no_components', [50, 75, 100 ]),
    'loss': hp.choice('loss', ['bpr', 'warp']),
    'learning_rate': hp.uniform('learning_rate', 0, 0.1),
    'item_alpha': hp.uniform('item_alpha', 0, 1),
    'user_alpha': hp.uniform('user_alpha', 0, 1)
}

In [ ]:
def hyperparametr_tuning(params):
    model = LightFM(**params, random_state=42)
    
    model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15, 
          num_threads=4,
          verbose=False)
    
    test_precision = precision_at_k(model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()
    
    return {'loss': -test_precision, 'status': STATUS_OK}

In [ ]:
trial = Trials()

best = fmin(
    fn=hyperparametr_tuning,
    space=space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trial
)

print(f'Best {best}')

In [ ]:
best_model = LightFM(no_components=50,
                loss='warp',
                learning_rate=0.03, 
                item_alpha=0.84,
                user_alpha=0.67, 
                random_state=42)

In [ ]:
best_model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15, 
          num_threads=4,
          verbose=False)

In [ ]:
precision_at_k(best_model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()